In [91]:
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from geopy.geocoders import Nominatim
from shapely.geometry import Polygon, Point
import geopandas as gpd
from tqdm import tqdm
import regex as re
from pathlib import Path
import json


In [159]:
DATA_DIR = Path("data_bomen")
INCIDENT_DATA_PATH = DATA_DIR / 'Stormdata & FireStations geadresseerd.csv'
TREE_DATA_PATH =  DATA_DIR / "BOMEN_DATA.csv"
TREE_DATA_WITH_ZIP_PATH = DATA_DIR / "BOMEN_DATA_WITH_ZIP.csv"
ZIPCODE_JSON_PATH = DATA_DIR / "zipcodes_boxes.json"
TREE_DATA_CLEAN_PATH = DATA_DIR / "tree_geo_data_clean.csv"
GRID_DATA_PATH = DATA_DIR / "grid_enriched.csv"
INCIDENTS_WEATHER_PATH = DATA_DIR / "incidents_weather.csv"
INCIDENTS_WEATHER_GEO_PATH = DATA_DIR / "incidents_weather_geo.csv"

ZIP_KEY = "Zipcode"
ZIP4_KEY = "Zip4"

TREE_COLUMNS = [
    "id",
    "soortnaamKort",        # andere soortnamen??
    "boomhoogte",
    "stamdiameter",
    "jaarVanAanleg",
    "typeObject",
    "standplaatsGedetailleerd",
    'SDVIEW',
    "RADIUS",
    "location",
    "grid_id",
]

MAP_BOOMHOOGTE = {
    'a. tot 6 m.' : "0-6",
    'b. 6 tot 9 m.': "6-9",
    'c. 9 tot 12 m.': "9-12",
    'd. 12 tot 15 m.': "12-15",
    'e. 15 tot 18 m.': "15-18",
    'f. 18 tot 24 m.': "18-25",
    'g. 24 m. en hoger': "24",
    'q. Niet van toepassing': "hQ"
}

MAP_STAMDIAMETER = {
    '0,1 tot 0,2 m.': "0.1-0.2",
    '0,2 tot 0,3 m.' : "0.2-0.3",
    '0,3 tot 0,5 m.': "0.3-0.5",
    '0,5 tot 1 m.': "0.5-1.0",
    '1,0 tot 1,5 m.': "1.0-1.5",
    '1,5 m. en grot': "1.5",
    'Onbekend': "dQ",
}

SERVICE_AREAS_OUT_OF_SCOPE = [
    "Amstelveen",
    "Aalsmeer",
    "Uithoorn"
]

RF_INCIDENT_COLUMNS = [
    "Incident_ID",
    "Service_Area",
    "grid_id",
    "Date",
    "Hour",
    "temperature_2m",
    "relative_humidity_2m",
    "dew_point_2m",
    "apparent_temperature",
    "precipitation",
    "rain",
    "snowfall",
    "snow_depth",
    "weather_code",
    "pressure_msl",
    "surface_pressure",
    "wind_speed_10m",
    "wind_direction_10m",
    "wind_gusts_10m",
    "soil_temperature_0_to_7cm",
    "soil_temperature_7_to_28cm",
    "soil_temperature_28_to_100cm",
    "soil_temperature_100_to_255cm",
    "soil_moisture_0_to_7cm",
    "soil_moisture_7_to_28cm",
    "soil_moisture_28_to_100cm",
    "soil_moisture_100_to_255cm",
]

RF_TREE_COLUMNS = [
    "tree_id",
    "grid_id",
    "soortnaamKort",
    "boomhoogte",
    "stamdiameter",
    "jaarVanAanleg",
    "typeObject",
    "standplaatsGedetailleerd",
]

RF_GRID_COLUMNS = [
    "grid_id",
    "has_tree",
    "avg_height",
    "avg_diameter",
    'avg_year',
    'Fraxinus', 
    'Salix', 
    'Alnus', 
    'Quercus', 
    'Tilia', 
    'Acer',
    'Populus', 
    'Betula', 
    'Prunus', 
    'Platanus', 
    'Malus', 
    'Robinia',
    'Crataegus', 
    'Ulmus', 
    'Carpinus', 
    'Overig', 
    'Onbekend'
]

In [93]:
# read storm_data
df = pd.read_csv(INCIDENT_DATA_PATH, sep=",", encoding="utf-8")
df = df.drop(['Unnamed: 0'], axis=1)
df = df.set_index('Incident_ID')

# read tree data
trees = pd.read_csv(TREE_DATA_PATH, sep=",", encoding="utf-8")

In [94]:
incidents_weather_df = pd.read_csv(INCIDENTS_WEATHER_PATH, sep=",", encoding="utf-8")
# Filter on areas in scope
incidents_weather_df = incidents_weather_df[~incidents_weather_df.Service_Area.isin(SERVICE_AREAS_OUT_OF_SCOPE)]


In [95]:
#Extracts the first 4 digits of zipcode '1010 AA' --> '1010' 
def extract_zip_4(
    df,
    zip_col,
    new_col,
):
    # Strips off last two number from zipcode (1010 AA - > 1010)
    pattern = r'\d{4}'
    df[new_col] = [re.match(pattern, zip_code)[0] if isinstance(zip_code, str) else None for zip_code in df[zip_col].values]
    return df


def add_zipcode(
    df
):
    # Adds zipcode based on lat and long coordinate values
    geolocator = Nominatim(user_agent="my_request")
    for index, row in tqdm(df.iterrows(), total=len(df), desc="Processing"):
        Latitude = row['LAT']
        Longitude = row['LNG']
        
        location = geolocator.reverse(f"{Latitude},{Longitude}")
        address = location.raw['address']
    
        zipcode = str(address.get('postcode'))  
        
        df.at[index, 'Zipcode'] = zipcode
    return df

In [96]:
df_tree_incidents = df[df["Damage_Type"]=="Tree"]
df_tree_incidents = extract_zip_4(df=df_tree_incidents, zip_col=ZIP_KEY, new_col=ZIP4_KEY)
df_tree_incidents

/var/folders/q3/3z4n8f9n2v9fphcv29rfd_4w0000gn/T/ipykernel_56301/3130433912.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Incident_Starttime,Incident_Endtime,Incident_Duration,Incident_Priority,Service_Area,Damage_Type,Municipality,LON,LAT,Day,Month,Year,City,Suburb,Residential,Road,Zipcode,Zip4
Incident_ID,,,,,,,,,,,,,,,,,,
511,08:25:18,10:00:56,95.633,2.0,Amstelveen,Tree,Amstelveen,4.838685,52.281552,7,9,2018,NaN,NaN,NaN,Bouwerij,1185 XW,1185
738,16:46:38,18:00:55,74.283,2.0,Victor,Tree,Amsterdam,4.930968,52.359724,10,9,2018,Amsterdam,Oost,Dapperbuurt,Linnaeusplantsoen,1093 KL,1093
1493,06:59:05,08:21:25,82.333,2.0,Amstelveen,Tree,Amstelveen,4.879741,52.301365,21,9,2018,NaN,NaN,NaN,Oranjebaan,1183 PL,1183
1502,08:29:22,10:06:43,97.350,2.0,Dirk,Tree,Amsterdam,4.870656,52.355189,21,9,2018,Amsterdam,Zuid,Museumkwartier,Van Breestraat,1071 ZM,1071
1507,09:25:55,11:43:54,137.983,2.0,Teunis,Tree,Amsterdam,4.832333,52.380748,21,9,2018,Amsterdam,Nieuw-West,NaN,M.C. Addicksstraat,1063 VZ,1063
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
436479,18:47:00,20:36:09,109.150,5.0,Victor,Tree,Amsterdam,4.944491,52.352064,31,7,2023,Amsterdam,Oost,Middenmeer,Rutherfordstraat,1098 TM,1098
436603,14:10:34,15:13:30,62.933,2.0,Osdorp,Tree,Amsterdam,4.824533,52.376163,2,8,2023,Amsterdam,Nieuw-West,NaN,Louis Couperusstraat,1064 CE,1064
436834,03:38:27,06:29:11,170.733,5.0,Nico,Tree,Amsterdam,4.928197,52.390568,6,8,2023,Amsterdam,Noord,Tuindorp Buiksloot,Nieuwendammerdijk,1025 LP,1025


In [97]:
#Map zips to coordinates
def map_zip_to_crs(
    df
):
    geolocator = Nominatim(user_agent="my_request")
    zipcodes = df['Zipcode'].unique()
    zip_dict = {}
    for code in tqdm(zipcodes, desc="Processing ZIP codes"):
        result = geolocator.geocode(query={'postalcode': code, 'country': 'Nederland'}, addressdetails=True)
        if result is not None and 'boundingbox' in result.raw:
            zip_dict[code] = result.raw['boundingbox']

    with open(ZIPCODE_JSON_PATH, 'w') as f:
        json.dump(zip_dict, f)



In [98]:
def find_zip(
    point,
    zip_dict
):
    lon, lat = float(point[0]), float(point[1])
    for zip_code, box in zip_dict.items():
        min_lat, max_lat, min_lon, max_lon = map(float, box)
        if min_lat <= lat <= max_lat and min_lon <= lon <= max_lon:
            return zip_code
    return None

def map_point_to_zip(
    df,
    zip_dict
):
    for i, row in tqdm(df.iterrows()):
        p = (row['LNG'], row["LAT"])
        zip_code = find_zip(p, zip_dict)
        df.at[i, 'Zipcode'] = zip_code
    return df

In [99]:
# map trees to zipcode

# takes a while only run if necessary
# map_zip_to_crs(df=df_tree_incidents)

# TODO: Werkt niet goed, ergens zit een probleem met de zips


with open(ZIPCODE_JSON_PATH, 'r') as f:
    zip_dict = json.load(f)

trees = map_point_to_zip(trees, zip_dict)
trees = extract_zip_4(df=trees, zip_col='Zipcode', new_col='Zip4')

trees.to_csv(TREE_DATA_WITH_ZIP_PATH, sep=",", encoding="utf-8")

0it [00:00, ?it/s]/var/folders/q3/3z4n8f9n2v9fphcv29rfd_4w0000gn/T/ipykernel_56301/124125364.py:19: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '1185 XW' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.

267974it [00:11, 24311.10it/s]


### Grid - Tree mapping

Nog niet perfect maar methode werkt.

In [100]:

geolocator = Nominatim(user_agent="my_geocoder")

# Get coordinates for Amsterdam
location = geolocator.geocode("Amsterdam, Netherlands")
amsterdam_lat, amsterdam_lon = location.latitude, location.longitude

amsterdam_bbox = (52.26618, 4.64663, 52.475115999999994, 5.150491999999999)

# Define grid size in meters
grid_size = 100

# Calculate grid bounds
lat_step = grid_size / 111000  # 1 degree of latitude is approximately 111 kilometers
lon_step = (grid_size / 111000) / np.cos(np.radians(amsterdam_lat))  # Correct for latitude

grid_polygons = []
for lat in np.arange(amsterdam_bbox[0], amsterdam_bbox[2], lat_step):
    for lon in np.arange(amsterdam_bbox[1], amsterdam_bbox[3], lon_step):
        polygon = Polygon([
            (lon, lat),
            (lon + lon_step, lat),
            (lon + lon_step, lat + lat_step),
            (lon, lat + lat_step),
            (lon, lat),
        ])
        grid_polygons.append(polygon)

grid_gdf = gpd.GeoDataFrame(geometry=grid_polygons, crs="EPSG:4326")

In [101]:
# create gdf from trees
tree_gdf = gpd.GeoDataFrame(trees, geometry=gpd.points_from_xy(trees['LNG'], trees['LAT']), crs="EPSG:4326")
# create gdf from indicents
incident_gdf = gpd.GeoDataFrame(incidents_weather_df, geometry=gpd.points_from_xy(incidents_weather_df['LON'], incidents_weather_df['LAT']), crs="EPSG:4326")
#join with grid gdf
tree_gdf = gpd.sjoin(tree_gdf, grid_gdf, how="left", op="within")
incident_gdf = gpd.sjoin(incident_gdf, grid_gdf, how="left", op="within")


/Users/Reimer/opt/anaconda3/envs/fundamentals-data-science/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3466: FutureWarning:

The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.

/Users/Reimer/opt/anaconda3/envs/fundamentals-data-science/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3466: FutureWarning:

The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.



In [102]:
#clean up gdf
tree_gdf = tree_gdf.rename(columns={"index_right" : "grid_id", "geometry" : "location"})
incident_gdf = incident_gdf.rename(columns={"index_right" : "grid_id", "geometry" : "location"})

#rename categories in new col, in place so only run once
tree_gdf['boomhoogte'] = [MAP_BOOMHOOGTE[klasse] if not klasse is np.nan else np.nan for klasse in tree_gdf.boomhoogteklasseActueel.values]
tree_gdf['stamdiameter'] = [MAP_STAMDIAMETER[klasse] if not klasse is np.nan else np.nan for klasse in tree_gdf.stamdiameterklasse.values]

# get rid of unnecessary columns
tree_gdf = tree_gdf[TREE_COLUMNS]

# save to new df
tree_gdf.to_csv(TREE_DATA_CLEAN_PATH, sep=",", encoding="utf-8")

In [103]:
incident_gdf

,Incident_ID,Date,Incident_Starttime,Hour,Date_time,Incident_Endtime,Incident_Duration,Incident_Priority,Service_Area,Municipality,...,soil_moisture_0_to_7cm,soil_moisture_7_to_28cm,soil_moisture_28_to_100cm,soil_moisture_100_to_255cm,Suburb,Residential,Road,Zipcode,location,grid_id
1,738,2018-09-10,16:46:38,16,2018/09/10 16:46,18:00:55,01:14:17,2.0,Victor,Amsterdam,...,0.527,0.484,0.459,0.617,Oost,Dapperbuurt,Linnaeusplantsoen,1093 KL,POINT (4.93097 52.35972),35418
3,1502,2018-09-21,08:29:22,8,2018/09/21 08:29,10:06:43,01:37:21,2.0,Dirk,Amsterdam,...,0.580,0.548,0.494,0.623,Zuid,Museumkwartier,Van Breestraat,1071 ZM,POINT (4.87066 52.35519),33667
4,1507,2018-09-21,09:25:55,9,2018/09/21 09:25,11:43:54,02:17:59,2.0,Teunis,Amsterdam,...,0.521,0.498,0.465,0.619,Nieuw-West,NaN,M.C. Addicksstraat,1063 VZ,POINT (4.83233 52.38075),43559
5,1548,2018-09-21,17:53:27,17,2018/09/21 17:53,20:14:56,02:21:29,2.0,Pieter,Amsterdam,...,0.582,0.546,0.494,0.623,NaN,NaN,NaN,NaN,POINT (4.83505 52.36686),38089
7,1552,2018-09-21,18:25:03,18,2018/09/21 18:25,20:15:21,01:50:18,2.0,Willem,Amsterdam,...,0.593,0.546,0.494,0.623,Zuid,NaN,Amsteldijk,1079 LK,POINT (4.90538 52.33572),26509
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2420,443469,2023-11-02,19:06:07,19,2023/11/02 19:06,19:06:38,00:00:31,2.0,Weesp,Amsterdam,...,0.732,0.737,0.724,0.635,NaN,NaN,NaN,NaN,POINT (5.07611 52.29911),12603
2421,443472,2023-11-02,19:45:17,19,2023/11/02 19:45,20:27:36,00:42:19,2.0,Willem,Amsterdam,...,0.740,0.743,0.708,0.632,NaN,NaN,NaN,NaN,POINT (4.90822 52.32678),23091
2422,443474,2023-11-02,19:53:02,19,2023/11/02 19:53,23:41:51,03:48:49,2.0,Pieter,Amsterdam,...,0.740,0.743,0.708,0.632,NaN,NaN,NaN,NaN,POINT (4.82213 52.34233),28846
2423,443476,2023-11-02,20:00:19,20,2023/11/02 20:00,20:52:15,00:51:56,2.0,Osdorp,Amsterdam,...,0.740,0.742,0.709,0.632,NaN,NaN,NaN,NaN,POINT (4.78724 52.36278),36689


In [104]:
import plotly.express as px
def plot_spacial_data(
    grid_gdf,
    tree_gdf,
    incident_gdf,
    plot_trees = True,
    plot_incidents = True
):
    # Create a plotly figure
    fig = px.choropleth_mapbox(grid_gdf, 
                                geojson=grid_gdf.geometry.__geo_interface__, 
                                locations=grid_gdf.index,
                                mapbox_style="open-street-map",
                                zoom=11, center={"lat": amsterdam_bbox[0], "lon": amsterdam_bbox[1]},
                                opacity=0.1,
                                )

    if plot_trees:
        # Add scatter plot for tree points
        fig.add_scattermapbox(
            lat=tree_gdf.location.y,
            lon=tree_gdf.location.x,
            mode='markers',
            marker=dict(
                size=4,
                color='green',
                opacity=0.7,
            ),
            text=tree_gdf['id'].astype(str),
            name='Trees'
        )
    if plot_incidents:
        # Add scatter plot for tree points
        fig.add_scattermapbox(
            lat=incident_gdf.location.y,
            lon=incident_gdf.location.x,
            mode='markers',
            marker=dict(
                size=4,
                color='red',
                opacity=0.7,
            ),
            text=incident_gdf.Incident_ID.astype(str),
            name='Incidents'
        )
    # Update the layout to make it interactive
    fig.update_layout(mapbox_style="carto-positron")
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    fig.show()
    

In [105]:
incident_gdf

,Incident_ID,Date,Incident_Starttime,Hour,Date_time,Incident_Endtime,Incident_Duration,Incident_Priority,Service_Area,Municipality,...,soil_moisture_0_to_7cm,soil_moisture_7_to_28cm,soil_moisture_28_to_100cm,soil_moisture_100_to_255cm,Suburb,Residential,Road,Zipcode,location,grid_id
1,738,2018-09-10,16:46:38,16,2018/09/10 16:46,18:00:55,01:14:17,2.0,Victor,Amsterdam,...,0.527,0.484,0.459,0.617,Oost,Dapperbuurt,Linnaeusplantsoen,1093 KL,POINT (4.93097 52.35972),35418
3,1502,2018-09-21,08:29:22,8,2018/09/21 08:29,10:06:43,01:37:21,2.0,Dirk,Amsterdam,...,0.580,0.548,0.494,0.623,Zuid,Museumkwartier,Van Breestraat,1071 ZM,POINT (4.87066 52.35519),33667
4,1507,2018-09-21,09:25:55,9,2018/09/21 09:25,11:43:54,02:17:59,2.0,Teunis,Amsterdam,...,0.521,0.498,0.465,0.619,Nieuw-West,NaN,M.C. Addicksstraat,1063 VZ,POINT (4.83233 52.38075),43559
5,1548,2018-09-21,17:53:27,17,2018/09/21 17:53,20:14:56,02:21:29,2.0,Pieter,Amsterdam,...,0.582,0.546,0.494,0.623,NaN,NaN,NaN,NaN,POINT (4.83505 52.36686),38089
7,1552,2018-09-21,18:25:03,18,2018/09/21 18:25,20:15:21,01:50:18,2.0,Willem,Amsterdam,...,0.593,0.546,0.494,0.623,Zuid,NaN,Amsteldijk,1079 LK,POINT (4.90538 52.33572),26509
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2420,443469,2023-11-02,19:06:07,19,2023/11/02 19:06,19:06:38,00:00:31,2.0,Weesp,Amsterdam,...,0.732,0.737,0.724,0.635,NaN,NaN,NaN,NaN,POINT (5.07611 52.29911),12603
2421,443472,2023-11-02,19:45:17,19,2023/11/02 19:45,20:27:36,00:42:19,2.0,Willem,Amsterdam,...,0.740,0.743,0.708,0.632,NaN,NaN,NaN,NaN,POINT (4.90822 52.32678),23091
2422,443474,2023-11-02,19:53:02,19,2023/11/02 19:53,23:41:51,03:48:49,2.0,Pieter,Amsterdam,...,0.740,0.743,0.708,0.632,NaN,NaN,NaN,NaN,POINT (4.82213 52.34233),28846
2423,443476,2023-11-02,20:00:19,20,2023/11/02 20:00,20:52:15,00:51:56,2.0,Osdorp,Amsterdam,...,0.740,0.742,0.709,0.632,NaN,NaN,NaN,NaN,POINT (4.78724 52.36278),36689


### Enrich grid gdf

In [106]:
tree_gdf[tree_gdf.grid_id == 31296]

''' 
grid gdf cols
grid_id, geometery, has_tree, [tree_counts], [averages] 
'''

def convert_cat_to_avg(
    cat_values,
    delimeter = "-"
):
    ''' 
    Converts categorical values to means of type float
    Splits cat values on delimter, computes the mean for each cat
    Returns mean of all means of the categories
    '''
    means = []
    for cat in cat_values:
        if not isinstance(cat, str):
            continue
        if not delimeter in cat:
            continue
        vals = cat.split(delimeter)
        means.append(np.mean([float(val) for val in vals]))
    m = round(np.mean(means), 3)
    return 0 if np.isnan(m) else m


def enrich_grid_df(
    grid_gdf,
    tree_gdf
):
    for i in grid_gdf.index:
        tree_sub_df = tree_gdf[tree_gdf.grid_id == i]
        if len(tree_sub_df)>0:
            # Compute and add averages for height, diameter and year
            grid_gdf.at[i, "avg_height"] = convert_cat_to_avg(tree_sub_df.boomhoogte.values)
            grid_gdf.at[i, "avg_diameter"] = convert_cat_to_avg(tree_sub_df.stamdiameter.values)
            grid_gdf.at[i, "avg_year"] = round(np.mean(tree_sub_df.jaarVanAanleg.values), 3)
            # Add soortnaam counts
            for name, count in tree_sub_df.soortnaamKort.value_counts().items():
                grid_gdf.at[i, "has_tree"] = True
                grid_gdf.at[i, name] = count
        else:
            grid_gdf.at[i, "has_tree"] = False


enrich_grid_df(grid_gdf=grid_gdf, tree_gdf=tree_gdf)

/var/folders/q3/3z4n8f9n2v9fphcv29rfd_4w0000gn/T/ipykernel_56301/3125811041.py:45: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'False' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.

/Users/Reimer/opt/anaconda3/envs/fundamentals-data-science/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning:

Mean of empty slice.

/Users/Reimer/opt/anaconda3/envs/fundamentals-data-science/lib/python3.11/site-packages/numpy/core/_methods.py:129: RuntimeWarning:

invalid value encountered in scalar divide

/Users/Reimer/opt/anaconda3/envs/fundamentals-data-science/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning:

Mean of empty slice.

/Users/Reimer/opt/anaconda3/envs/fundamentals-data-science/lib/python3.11/site-packages/numpy/core/_methods.py:129: RuntimeWarning:

invalid value encountered in scalar divide

/Users/Reimer/opt/anaconda3/e

In [107]:
incident_gdf.to_csv(INCIDENTS_WEATHER_GEO_PATH, sep=",", encoding="utf-8")

In [108]:
grid_gdf = grid_gdf.fillna(0)
grid_gdf[grid_gdf.has_tree == True]

grid_gdf.to_csv(GRID_DATA_PATH, sep=",", encoding="utf-8")

/var/folders/q3/3z4n8f9n2v9fphcv29rfd_4w0000gn/T/ipykernel_56301/3873820944.py:1: DeprecationWarning:

ExtensionArray.fillna added a 'copy' keyword in pandas 2.1.0. In a future version, ExtensionArray subclasses will need to implement this keyword or an exception will be raised. In the interim, the keyword is ignored by GeometryArray.



In [ ]:
# plot_spacial_data(grid_gdf=grid_gdf, tree_gdf=tree_gdf, incident_gdf=incident_gdf)

### Create train / test sets

In [114]:
def display_full_df(
    df
):
    pd.set_option('display.max_rows', None)
    print(df.to_string(index=False))
    pd.reset_option('display.max_rows')

In [157]:
incident_gdf

,Incident_ID,Date,Incident_Starttime,Hour,Date_time,Incident_Endtime,Incident_Duration,Incident_Priority,Service_Area,Municipality,...,soil_moisture_0_to_7cm,soil_moisture_7_to_28cm,soil_moisture_28_to_100cm,soil_moisture_100_to_255cm,Suburb,Residential,Road,Zipcode,location,grid_id
1,738,2018-09-10,16:46:38,16,2018/09/10 16:46,18:00:55,01:14:17,2.0,Victor,Amsterdam,...,0.527,0.484,0.459,0.617,Oost,Dapperbuurt,Linnaeusplantsoen,1093 KL,POINT (4.93097 52.35972),35418
3,1502,2018-09-21,08:29:22,8,2018/09/21 08:29,10:06:43,01:37:21,2.0,Dirk,Amsterdam,...,0.580,0.548,0.494,0.623,Zuid,Museumkwartier,Van Breestraat,1071 ZM,POINT (4.87066 52.35519),33667
4,1507,2018-09-21,09:25:55,9,2018/09/21 09:25,11:43:54,02:17:59,2.0,Teunis,Amsterdam,...,0.521,0.498,0.465,0.619,Nieuw-West,NaN,M.C. Addicksstraat,1063 VZ,POINT (4.83233 52.38075),43559
5,1548,2018-09-21,17:53:27,17,2018/09/21 17:53,20:14:56,02:21:29,2.0,Pieter,Amsterdam,...,0.582,0.546,0.494,0.623,NaN,NaN,NaN,NaN,POINT (4.83505 52.36686),38089
7,1552,2018-09-21,18:25:03,18,2018/09/21 18:25,20:15:21,01:50:18,2.0,Willem,Amsterdam,...,0.593,0.546,0.494,0.623,Zuid,NaN,Amsteldijk,1079 LK,POINT (4.90538 52.33572),26509
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2420,443469,2023-11-02,19:06:07,19,2023/11/02 19:06,19:06:38,00:00:31,2.0,Weesp,Amsterdam,...,0.732,0.737,0.724,0.635,NaN,NaN,NaN,NaN,POINT (5.07611 52.29911),12603
2421,443472,2023-11-02,19:45:17,19,2023/11/02 19:45,20:27:36,00:42:19,2.0,Willem,Amsterdam,...,0.740,0.743,0.708,0.632,NaN,NaN,NaN,NaN,POINT (4.90822 52.32678),23091
2422,443474,2023-11-02,19:53:02,19,2023/11/02 19:53,23:41:51,03:48:49,2.0,Pieter,Amsterdam,...,0.740,0.743,0.708,0.632,NaN,NaN,NaN,NaN,POINT (4.82213 52.34233),28846
2423,443476,2023-11-02,20:00:19,20,2023/11/02 20:00,20:52:15,00:51:56,2.0,Osdorp,Amsterdam,...,0.740,0.742,0.709,0.632,NaN,NaN,NaN,NaN,POINT (4.78724 52.36278),36689


In [170]:
# Pick necessary columns

incident_sub_gdf = incident_gdf[RF_INCIDENT_COLUMNS]

grid_gdf['grid_id'] = grid_gdf.index
grid_sub_gdf = grid_gdf[RF_GRID_COLUMNS]

tree_gdf = tree_gdf.rename(columns={"id" : "tree_id"})
tree_sub_gdf = tree_gdf[RF_TREE_COLUMNS]

In [171]:
incident_sub_gdf

,Incident_ID,Service_Area,grid_id,Date,Hour,temperature_2m,relative_humidity_2m,dew_point_2m,apparent_temperature,precipitation,...,wind_direction_10m,wind_gusts_10m,soil_temperature_0_to_7cm,soil_temperature_7_to_28cm,soil_temperature_28_to_100cm,soil_temperature_100_to_255cm,soil_moisture_0_to_7cm,soil_moisture_7_to_28cm,soil_moisture_28_to_100cm,soil_moisture_100_to_255cm
1,738,Victor,35418,2018-09-10,16,17.704500,71.116135,12.4045,15.858986,0.0,...,255.96373,37.079998,18.3045,16.8545,16.5045,13.704500,0.527,0.484,0.459,0.617
3,1502,Dirk,33667,2018-09-21,8,14.935000,75.217130,10.5850,10.086021,0.1,...,258.57294,70.920000,16.8850,17.6850,15.8850,13.785001,0.580,0.548,0.494,0.623
4,1507,Teunis,43559,2018-09-21,9,15.080501,64.915630,8.5305,9.437960,0.1,...,261.38443,64.799995,16.9305,17.7305,15.9305,13.880500,0.521,0.498,0.465,0.619
5,1548,Pieter,38089,2018-09-21,17,12.274000,79.467766,8.8240,7.439817,1.5,...,232.06667,66.240000,15.8740,17.4740,15.9240,13.874000,0.582,0.546,0.494,0.623
7,1552,Willem,26509,2018-09-21,18,12.461000,81.939240,9.4610,8.262999,1.0,...,244.75950,72.720000,15.2110,17.3610,15.9110,13.861000,0.593,0.546,0.494,0.623
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2420,443469,Weesp,12603,2023-11-02,19,10.695499,68.317810,5.0955,3.138343,0.0,...,199.78117,82.079994,10.4955,11.3455,12.8455,14.045500,0.732,0.737,0.724,0.635
2421,443472,Willem,23091,2023-11-02,19,9.950000,74.620030,5.6500,2.380546,0.0,...,197.40940,81.720000,9.7000,11.1500,12.8500,14.000000,0.740,0.743,0.708,0.632
2422,443474,Pieter,28846,2023-11-02,19,9.924000,74.615500,5.6240,2.348835,0.0,...,197.40940,81.720000,9.6740,11.1240,12.8240,13.974000,0.740,0.743,0.708,0.632
2423,443476,Osdorp,36689,2023-11-02,20,9.500000,77.172410,5.7000,2.024211,0.3,...,199.62234,80.640000,9.6500,11.1000,12.8500,14.000000,0.740,0.742,0.709,0.632


In [172]:
positive_samples = grid_sub_gdf.merge(incident_sub_gdf, on='grid_id', how='inner')

In [173]:
positive_samples.to_csv("positive_samples.csv", sep=",", encoding="utf-8")

In [178]:
p_sample_info = positive_samples[['Incident_ID', 'grid_id', 'Date', 'Hour']]

In [180]:
p_sample_info.to_csv("p_sample_info.csv", sep=",", encoding="utf-8")